In [ ]:
# ✅ 1. 라이브러리 로드
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoConfig
import torch
import re

# ✅ 2. 모델 경로 설정
model_path = "./KO_results"  # 학습 결과가 저장된 디렉토리

# ✅ 3. config.json이 없을 경우 대비 – 원본 모델에서 복사
try:
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    model = AutoModelForSequenceClassification.from_pretrained(model_path).to("mps")
except OSError:
    print("config.json이 없어 원본 모델에서 복사합니다.")
    from transformers import AutoConfig
    config = AutoConfig.from_pretrained("beomi/KcBERT-base", num_labels=2)
    config.save_pretrained(model_path)
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    model = AutoModelForSequenceClassification.from_pretrained(model_path, config=config).to("mps")

model.eval()

# ✅ 4. 텍스트 정제 함수
def clean_text(text):
    text = re.sub(r"[\\n\\r]", " ", text)
    text = re.sub(r"\\s+", " ", text)
    return text.strip()

# ✅ 5. 추론 함수
def predict_spam(text_list):
    results = []
    for text in text_list:
        cleaned = clean_text(text)
        inputs = tokenizer(
            cleaned,
            return_tensors="pt",
            padding="max_length",
            truncation=True,
            max_length=256
        ).to("mps")

        with torch.no_grad():
            outputs = model(**inputs)
            logits = outputs.logits
            prediction = torch.argmax(logits, dim=1).item()
            label = "SPAM" if prediction == 1 else "HAM"
            confidence = torch.softmax(logits, dim=1)[0][prediction].item()

        results.append((text, label, round(confidence, 4)))
    return results

# ✅ 6. 테스트 문장
samples = [
    # 📧 일반 업무용
    "금주 회의록을 첨부드립니다. 확인 부탁드립니다.",
    "새 프로젝트 일정은 아래와 같습니다.",
    "다음주 휴가 일정 공유드립니다.",
    "회의실 예약이 완료되었습니다.",
    "재무팀에 인보이스를 전송해주세요.",
    "업무 보고서를 내일까지 제출해주세요.",

    # 💰 광고/스팸
    "지금 가입하면 100% 당첨의 기회!",
    "단 3일만! 무료로 체험하세요!",
    "최신 스마트폰, 공짜로 드립니다!",
    "무료 포인트 적립! 오늘 안 받으면 손해!",
    "신규 회원 한정, 스타벅스 기프티콘 증정!",
    "집에서도 하루 30분으로 500만원 수익!",
    "이 링크를 클릭하고 비밀 쿠폰을 받으세요!",
    "미청구 보험금 확인해보셨나요?",
    "소지하고 계신 카드가 당첨되었습니다!",
    "이 메일을 10명에게 보내면 선물이 도착합니다!",

    # 🛒 쇼핑/이벤트
    "11번가 타임딜, 지금 시작합니다!",
    "카카오쇼핑 3,000원 쿠폰이 도착했어요.",
    "오늘은 장바구니 쿠폰이 자동 적용됩니다.",
    "고객님을 위한 맞춤 상품을 준비했어요.",
    "이번 주 주말특가! 최대 70% 할인!",

    # 🔐 보안/피싱 의심
    "귀하의 계정에서 로그인 시도가 감지되었습니다.",
    "보안 강화를 위해 비밀번호를 변경해주세요.",
    "신원 확인이 필요합니다. 여기를 클릭하세요.",
    "이체 오류 발생. 계좌 인증이 필요합니다.",

    # 💬 일상 메시지
    "점심 뭐 먹을래?",
    "오늘 날씨 너무 좋다!",
    "주말에 영화 보러 갈래?",
    "집 앞에 도착했어~",
    "다음 주 생일 파티 기대돼!"
]


# ✅ 7. 결과 출력
results = predict_spam(samples)
for text, label, conf in results:
    print(f"\n📨 \"{text}\"\n➜ 예측: {label} (신뢰도: {conf})")


/Users/tilon/Desktop/Tilon/myvenv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/tilon/Desktop/Tilon/myvenv/lib/python3.13/site-packages/huggingface_hub/file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at beomi/KcBERT-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/Users/tilon/Desktop/Tilon/myvenv/lib/python3.13/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of b

'NoneType' object has no attribute 'cadam32bit_grad_fp32'

📨 "오늘 오후 2시에 회의가 있습니다."
➜ 예측: HAM (신뢰도: 0.999)

📨 "무료쿠폰! 지금 클릭하고 받아가세요!"
➜ 예측: SPAM (신뢰도: 0.9987)

📨 "사내 복지 변경사항을 공유드립니다."
➜ 예측: HAM (신뢰도: 0.9991)

📨 "365일 다이어트 비법, 단 3일만 공개!"
➜ 예측: SPAM (신뢰도: 0.9983)

📨 "이벤트에 당첨되셨습니다! 경품을 수령하세요."
➜ 예측: SPAM (신뢰도: 0.9976)

📨 "프로젝트 자료를 회람합니다."
➜ 예측: HAM (신뢰도: 0.9986)
